In [7]:
# =========================================
# 0. Install + import + font + drive + Config
# =========================================
!pip -q install gradio transformers torch pandas matplotlib scikit-learn tqdm

import os, pickle
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import gradio as gr

# 한글 폰트
try:
    !sudo apt-get install -y fonts-nanum >/dev/null
    !sudo fc-cache -fv >/dev/null
    !rm -rf ~/.cache/matplotlib
    plt.rc("font", family="NanumBarunGothic")
    plt.rcParams["axes.unicode_minus"] = False
except:
    pass

# Drive 마운트
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    print("[INFO] Google Drive mounted.")
except:
    print("[INFO] Drive mount skipped.")

class Config:
    PROJECT_DIR = "/content/drive/MyDrive/Projects/MindLog"
    DATA_DIR    = os.path.join(PROJECT_DIR, "processed")
    MODEL_DIR   = os.path.join(PROJECT_DIR, "models")

    MODEL_NAME  = "beomi/KcELECTRA-base-v2022"
    CKPT_NAME   = "best_multitask_model.bin"

    MAX_LEN     = 128
    DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs(Config.MODEL_DIR, exist_ok=True)
print("[INFO] Device:", Config.DEVICE)

Mounted at /content/drive
[INFO] Google Drive mounted.
[INFO] Device: cuda


In [11]:
# =========================================
# 1. Load label_map (id2emotion / id2situation)
# =========================================
label_map_path = os.path.join(Config.DATA_DIR, "label_map.pkl")
with open(label_map_path, "rb") as f:
    label_map = pickle.load(f)

id2emotion   = label_map["id2emotion"]
id2situation = label_map["id2situation"]

num_emo = len(label_map["emotion2id"])
num_sit = len(label_map["situation2id"])

emo_names = [id2emotion[i] for i in range(num_emo)]
sit_names = [id2situation[i] for i in range(num_sit)]

print("[INFO] num_emo:", num_emo, "num_sit:", num_sit)

[INFO] num_emo: 6 num_sit: 12


In [12]:
# =========================================
# 2. Model + tokenizer + ckpt load
# =========================================
from transformers import AutoTokenizer, AutoModel

class SentimentMultiTaskModel(nn.Module):
    def __init__(self, model_name, num_emo_classes, num_sit_classes):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(0.1)
        self.emo_classifier = nn.Linear(hidden, num_emo_classes)
        self.sit_classifier = nn.Linear(hidden, num_sit_classes)

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls = self.dropout(out.last_hidden_state[:, 0, :])
        return {
            "logits_emotion": self.emo_classifier(cls),
            "logits_situation": self.sit_classifier(cls),
        }

tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)

model = SentimentMultiTaskModel(Config.MODEL_NAME, num_emo, num_sit).to(Config.DEVICE)
ckpt_path = os.path.join(Config.MODEL_DIR, Config.CKPT_NAME)

state = torch.load(ckpt_path, map_location=Config.DEVICE)
model.load_state_dict(state)
model.eval()

print("[INFO] Loaded ckpt:", ckpt_path)

[INFO] Loaded ckpt: /content/drive/MyDrive/Projects/MindLog/models/best_multitask_model.bin


In [18]:
# ========================================================
# 3. Predict / History / Plots / Save
# ========================================================

def predict_probs(text: str):
    """return: emo_id, emo_conf, sit_id, sit_conf, emo_probs(np), sit_probs(np)"""
    text = (text or "").strip()
    enc = tokenizer(
        text,
        max_length=Config.MAX_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    enc = {k: v.to(Config.DEVICE) for k, v in enc.items()}

    with torch.no_grad():
        out = model(input_ids=enc["input_ids"], attention_mask=enc["attention_mask"])

    emo_probs_t = F.softmax(out["logits_emotion"], dim=-1)[0]
    sit_probs_t = F.softmax(out["logits_situation"], dim=-1)[0]

    emo_id = int(torch.argmax(emo_probs_t).item())
    sit_id = int(torch.argmax(sit_probs_t).item())
    emo_conf = float(torch.max(emo_probs_t).item())
    sit_conf = float(torch.max(sit_probs_t).item())

    return emo_id, emo_conf, sit_id, sit_conf, emo_probs_t.detach().cpu().numpy(), sit_probs_t.detach().cpu().numpy()


def append_turn(history, user_text, emo_id, emo_conf, sit_id, sit_conf, emo_probs, sit_probs):
    turn = len(history) + 1
    history.append({
        "turn": turn,
        "ts": datetime.now().isoformat(timespec="seconds"),
        "user": user_text,
        "emotion_id": emo_id,
        "emotion": id2emotion[emo_id],
        "emotion_conf": emo_conf,
        "situation_id": sit_id,
        "situation": id2situation[sit_id],
        "situation_conf": sit_conf,
        "emotion_probs": emo_probs.tolist(),
        "situation_probs": sit_probs.tolist(),
    })
    return history


def save_history_csv(history, filename_prefix="chat_history"):
    ts = datetime.now().strftime("%y%m%d_%H%M")  # yymmdd_hhmm
    filename = f"{filename_prefix}_{ts}.csv"
    path = os.path.join(Config.MODEL_DIR, filename)
    df = pd.DataFrame(history).drop(columns=["emotion_probs", "situation_probs"], errors="ignore")
    df.to_csv(path, index=False, encoding="utf-8-sig")
    return path

def format_current_pred(emo_id, emo_conf, sit_id, sit_conf):
    emo = id2emotion[emo_id]
    sit = id2situation[sit_id]
    return (
        f"### 현재 예측\n"
        f"- 감정: **{emo}**  (conf={emo_conf:.2f})\n"
        f"- 상황: **{sit}**  (conf={sit_conf:.2f})\n\n"
    )

In [19]:
# =========================================
# 4. Figures (Top-3 bar / timelines / confidence)
# =========================================

# -------------------------
# Plot: Top-3 bar charts
# -------------------------
def fig_topk_bar(probs, names, title, k=3):
    probs = np.array(probs)
    top_idx = probs.argsort()[::-1][:k][::-1]  # 낮->높
    top_names = [names[i] for i in top_idx]
    top_vals  = probs[top_idx]

    fig = plt.figure(figsize=(6, 3))
    plt.barh(top_names, top_vals)
    plt.xlim(0, 1.0)
    plt.title(title)
    plt.xlabel("probability")
    plt.grid(True, axis="x", alpha=0.2)
    plt.tight_layout()
    return fig


# -------------------------
# Plot: emotion timeline
# -------------------------
def fig_emotion_timeline(history):
    if not history:
        return None
    xs = [h["turn"] for h in history]
    ys = [h["emotion_id"] for h in history]
    conf = [h["emotion_conf"] for h in history]

    fig = plt.figure(figsize=(10, 3.2))
    plt.scatter(xs, ys, s=[c * 220 for c in conf])
    plt.yticks(range(len(emo_names)), emo_names)
    plt.xlabel("turn")
    plt.ylabel("emotion")
    plt.title("Emotion timeline (dot size = confidence)")
    plt.grid(True, axis="y", alpha=0.25)
    plt.tight_layout()
    return fig


# -------------------------
# Plot: situation timeline
# -------------------------
def fig_situation_timeline(history):
    if not history:
        return None
    xs = [h["turn"] for h in history]
    ys = [h["situation_id"] for h in history]
    conf = [h["situation_conf"] for h in history]

    fig = plt.figure(figsize=(10, 3.2))
    plt.scatter(xs, ys, s=[c * 220 for c in conf])
    plt.yticks(range(len(sit_names)), sit_names)
    plt.xlabel("turn")
    plt.ylabel("situation")
    plt.title("Situation timeline (dot size = confidence)")
    plt.grid(True, axis="y", alpha=0.25)
    plt.tight_layout()
    return fig


# -------------------------
# Plot: confidence over turns
# -------------------------
def fig_confidence(history):
    if not history:
        return None
    xs = [h["turn"] for h in history]
    emo_c = [h["emotion_conf"] for h in history]
    sit_c = [h["situation_conf"] for h in history]

    fig = plt.figure(figsize=(10, 3))
    plt.plot(xs, emo_c, marker="o", label="emotion conf")
    plt.plot(xs, sit_c, marker="o", label="situation conf")
    plt.ylim(0, 1.05)
    plt.xlabel("turn")
    plt.ylabel("confidence")
    plt.title("Confidence over turns")
    plt.grid(True, alpha=0.25)
    plt.legend()
    plt.tight_layout()
    return fig

In [20]:
# =========================================
# 5. Session Summary Card
# =========================================
def build_summary(history):
    if not history:
        return "### 세션 요약\n(아직 데이터 없음)"

    df = pd.DataFrame(history)

    # 최빈 감정/상황
    top_emo = df["emotion"].value_counts().index[0]
    top_sit = df["situation"].value_counts().index[0]

    # "최고 분노 turn": 분노 중 confidence 최고, 없으면 None
    anger_df = df[df["emotion"] == "분노"].copy()
    if len(anger_df) > 0:
        peak_anger_row = anger_df.sort_values("emotion_conf", ascending=False).iloc[0]
        peak_anger = f"turn {int(peak_anger_row['turn'])} (conf={peak_anger_row['emotion_conf']:.2f})"
    else:
        peak_anger = "없음"

    # 가장 확신 낮은 턴(감정/상황 중 낮은 쪽 기준)
    df["min_conf"] = df[["emotion_conf", "situation_conf"]].min(axis=1)
    low_row = df.sort_values("min_conf", ascending=True).iloc[0]
    lowest = f"turn {int(low_row['turn'])} (min_conf={low_row['min_conf']:.2f})"

    total = len(df)
    last_ts = df["ts"].iloc[-1]

    return (
        f"### 세션 요약\n"
        f"- 총 턴: **{total}**\n"
        f"- 최빈 감정: **{top_emo}**\n"
        f"- 최빈 상황: **{top_sit}**\n"
        f"- 최고 분노 턴: **{peak_anger}**\n"
        f"- 가장 애매한 턴: **{lowest}**\n"
        f"- 마지막 기록: `{last_ts}`"
    )

In [21]:
# =========================================
# 6. Gradio Dashboard (Silent Tracker)
# =========================================
def ui_append(user_text, history):
    user_text = (user_text or "").strip()
    if not user_text:
        df = pd.DataFrame(history) if history else pd.DataFrame()
        return (
            "", history,
            "텍스트를 입력해줘.",
            None, None,
            None, None, None,
            "### 세션 요약\n(아직 데이터 없음)",
            df, None
        )

    emo_id, emo_conf, sit_id, sit_conf, emo_probs, sit_probs = predict_probs(user_text)

    # history 업데이트
    history = append_turn(history, user_text, emo_id, emo_conf, sit_id, sit_conf, emo_probs, sit_probs)

    # 저장
    csv_path = save_history_csv(history, filename_prefix="chat_history")

    # UI 구성
    cur_pred_md = format_current_pred(emo_id, emo_conf, sit_id, sit_conf)
    emo_bar = fig_topk_bar(emo_probs, emo_names, "Top-3 Emotion Prob", k=3)
    sit_bar = fig_topk_bar(sit_probs, sit_names, "Top-3 Situation Prob", k=3)

    emo_tl  = fig_emotion_timeline(history)
    sit_tl  = fig_situation_timeline(history)
    conf_tl = fig_confidence(history)

    summary = build_summary(history)

    df = pd.DataFrame(history).drop(columns=["emotion_probs", "situation_probs"], errors="ignore")

    return "", history, cur_pred_md, emo_bar, sit_bar, emo_tl, sit_tl, conf_tl, summary, df, csv_path

def ui_reset():
    history = []
    df = pd.DataFrame()
    return (
        "", history,
        "세션이 초기화됐어.",
        None, None,
        None, None, None,
        "### 세션 요약\n(아직 데이터 없음)",
        df, None
    )

with gr.Blocks(title="MindLog Silent Tracker") as demo:
    gr.Markdown("# MindLog Silent Tracker\nLLM 없이 **분류 + 기록 + 시각화**만 하는 대시보드")

    state_history = gr.State([])

    with gr.Row():
        with gr.Column(scale=1):
            user_in = gr.Textbox(
                label="입력 텍스트",
                placeholder="여기에 오늘 감정/상황을 적고 Enter(또는 버튼)를 눌러줘",
                lines=4
            )
            with gr.Row():
                btn_add = gr.Button("추가(append)", variant="primary")
                btn_reset = gr.Button("세션 초기화", variant="secondary")

        with gr.Column(scale=2):
            cur_pred = gr.Markdown("아직 입력 없음.")
            with gr.Row():
                emo_bar_plot = gr.Plot(label="Top-3 Emotion")
                sit_bar_plot = gr.Plot(label="Top-3 Situation")

            emo_timeline_plot = gr.Plot(label="Emotion Timeline")
            sit_timeline_plot = gr.Plot(label="Situation Timeline")
            conf_plot = gr.Plot(label="Confidence")

            summary_md = gr.Markdown("### 세션 요약\n(아직 데이터 없음)")
            hist_table = gr.Dataframe(label="History", interactive=False)
            csv_file = gr.File(label="CSV 다운로드")

    # 버튼 클릭
    btn_add.click(
        fn=ui_append,
        inputs=[user_in, state_history],
        outputs=[user_in, state_history, cur_pred,
                 emo_bar_plot, sit_bar_plot,
                 emo_timeline_plot, sit_timeline_plot, conf_plot,
                 summary_md, hist_table, csv_file]
    )

    # Enter 제출 (Textbox submit)
    user_in.submit(
        fn=ui_append,
        inputs=[user_in, state_history],
        outputs=[user_in, state_history, cur_pred,
                 emo_bar_plot, sit_bar_plot,
                 emo_timeline_plot, sit_timeline_plot, conf_plot,
                 summary_md, hist_table, csv_file]
    )

    btn_reset.click(
        fn=ui_reset,
        inputs=[],
        outputs=[user_in, state_history, cur_pred,
                 emo_bar_plot, sit_bar_plot,
                 emo_timeline_plot, sit_timeline_plot, conf_plot,
                 summary_md, hist_table, csv_file]
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://027f13e7003e4c942d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://027f13e7003e4c942d.gradio.live
